# DonkeyCar 4.4 Training

Training DonkeyCar on GPU Instances in Google Colab
- Version 20230719(2023年7月19日)
- DonkeyCar version 4.4
- Tensorflow 2.2.0
- CUDA 10.1

## 概要
Google Colabを使用し、Donkey Carの学習を行います

上からコードをを実行することで、以下のことができます
- Google Colabの環境設定
- 学習
- 学習結果の取得  

## 更新履歴
20230719
- DonkeyCar 4.4
- CUDA 10.1
- Tensorflow 2.2.0


# ColabのOSを確認します

In [ ]:
!cat /etc/os-release

# 既存CUDAをアンインストール

In [ ]:
!apt --purge remove "cublas*" "cuda*"

## CUDA-10.1リポジトリ追加

In [ ]:
!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/7fa2af80.pub
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-keyring_1.1-1_all.deb
!dpkg -i cuda-keyring_1.1-1_all.deb

## CUDA-10.1インストール
10分くらい時間がかかります。

In [ ]:
!apt-get update
!apt-get install -y cuda-10.1

## CuDNN-7.6.5インストール

In [ ]:
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/libcudnn7_7.6.5.32-1+cuda10.1_amd64.deb
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/libcudnn7-dev_7.6.5.32-1+cuda10.1_amd64.deb
!dpkg -i \
      libcudnn7_7.6.5.32-1+cuda10.1_amd64.deb \
      libcudnn7-dev_7.6.5.32-1+cuda10.1_amd64.deb

# TensorFlowのインストール

TensorFlowのversionは、DonkeyCarのRaspberryPi3にはいっているバージョンと同じバージョンをインストールします。\
Google Colabはデフォルトのtensorflow/cuda/pipパッケージバージョンが更新されていきます。  \
Tensorflowは下位互換性を維持しないため、ColabにはRaspberry Pi版DonkeyCar標準のTensorFlow 2.2.0と同じバージョンのTensorflowを入れます。

Google Colabは古いTensorflowをサポートしなくなったため、pipで探すことができなくなりました。  \
そのため、Minicondaを利用してTensorflow 2.2.0をインストールします。

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))
!pip install -U pip
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==2.2.0 h5py==2.10.0 protobuf==3.19.6 numpy==1.19.5 pillow==8.4.0 cycler==0.11.0 decorator==4.4.2 python-dateutil==2.8.2 wcwidth==0.2.6 pandas==1.1.5 opencv-python==4.2.0.34

**RESTART RUNTIME**を1度実行してください。  \
リスタート後もインストールしたTensorflow 2.2.0がキープされます。 \
リスタート後はこの次から開始できます。

# RESTART後にPython PATHの再追加

In [ ]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

# TensorFlowのバージョン確認

In [ ]:
!pip freeze | grep -i -e tensorflow

In [ ]:
!python -c "import warnings;warnings.simplefilter(action='ignore', category=FutureWarning);import tensorflow as tf; print(tf.__version__)"

# DonkeyCarのインストール
Raspberry PiにインストールしてあるDonkeyCarとバージョンを合わせておきます。

In [ ]:
# DonkeyCarがimageioが見つからない場合に自動インストールするimageioのバージョンは、2023/03/02時点で最新の2.16.0です。これはnumpy1.20.0以上を必要とします。
# 2023/03/02時点ではgoogle colabで使えるnumpyは1.19.5までなので、imageioのバージョンを指定してインストールします。
!pip install imageio==2.15.0

In [ ]:
!git clone -b release_4_4 https://github.com/autorope/donkeycar.git
%cd donkeycar
!sed -i '/imgaug/d' setup.py
!sed -i '/pillow/d' setup.py
!pip install -e .

In [ ]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

# DonkeyCarのProjectを作成

In [ ]:
!donkey createcar --path /content/mycar

# data.zipのアップロードとデータの解凍
data.zip（DonkeyCarで集めた学習用データ）をアップロードします。\
ブラウザ経由でcolabにdata.zipをアップロードする方法と、\
Google Drive経由でcolabにdata.zipをアップロードする方法の2通りあります。\
ブラウザ経由の方法で通信速度が遅すぎる場合はGoogle Drive経由の方法を試してみてください。

# ブラウザ経由でdata.zipをアップロードする場合
data.zip（DonkeyCarで集めた学習用データ）をアップロードします。  
下記コードを実行すると、ファイルのアップロード フォームが出現します。

ファイルは`/content/mycar/`以下に送信され、data.zipの解凍を行っています。\
データセットのディレクトリ構造は、`/content/mycar/data/tub_1_xxxx`のようになります。

In [ ]:
%cd /content/mycar/

import os
import shutil
from google.colab import files

if(os.path.exists("/content/mycar/data.zip")):
   os.remove("/content/mycar/data.zip")
if(os.path.exists("/content/mycar/data")):
   shutil.rmtree("/content/mycar/data")

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = "/content/mycar/" + file_name
print("-----------------------------------------------------")
print("【処理結果】%sにデータをアップロードしました。" % file_path)
print("-----------------------------------------------------")
!unzip -o data.zip

# Google Drive経由でdata.zipをアップロードする場合
ブラウザ経由でデータをアップロードすると、とても遅いため、Google Drive経由でアップロードを早く完了させたい場合に使います。  
下記コードを実行して認証をおこなうことで、Google Driveにアップロードした自分のdata.zipファイルをGoogle Colabで利用できるようになります。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive' | grep data

Google Driveは、/content/drive/MyDriveにマウントされます。

data.zipファイルを/content/mycar/以下にコピーして、解凍をおこないます。

Google Driveの任意のフォルダにアップロードした場合は、'/content/drive/MyDrive/任意のフォルダ名/data.zip' のように置き換えてください。

In [ ]:
%cd /content/mycar/

import os
import shutil

if(os.path.exists("/content/mycar/data.zip")):
   os.remove("/content/mycar/data.zip")
if(os.path.exists("/content/mycar/data")):
   shutil.rmtree("/content/mycar/data")

!ls -l '/content/drive/MyDrive/data.zip'
!cp '/content/drive/MyDrive/data.zip' /content/mycar/data.zip
!unzip data.zip

#学習の開始

Colabの環境構築が出来たので、DonkeyCarの学習を行います。
<br>
<br>
学習の対象は、/content/mycar/data/に存在するフォルダ全部になります。  
学習時のtypeはlinearですが、学習が完了すると、mypilot.h5の他にmypilot.tfliteが作成されます。  

In [ ]:
import os
os.environ['LD_LIBRARY_PATH']='/usr/lib64-nvidia:/usr/local/cuda-10.2/targets/x86_64-linux/lib'

type = "linear"
trained_mode = "/content/mycar/models/mypilot.h5"

%cd /content/mycar
!python train.py --tub=data --model={trained_mode} --type={type}

# 学習結果の確認
DonkeyCarは学習済みモデルを解析して動画に出力する機能があります。

学習したモデルがどのようになったのかを確認します。

モデル毎にレイヤーが全く異なるため、makemovieを使えるモデルはlinearモデルのみです。  

### モデル可視化用パッケージをインストール

In [ ]:
!pip install moviepy==1.0.3 matplotlib==3.3.4
!pip install git+https://github.com/autorope/keras-vis.git

### 動画ファイルの作成


In [ ]:
!ls /content/mycar/data

In [ ]:
%cd /content/mycar
mp4_path = 'tub_movie.mp4'
!donkey makemovie --tub=data --model models/mypilot.h5 --type linear --salient --out=$mp4_path

### 動画のダウンロード
作成した解析動画をPCにダウンロードします。\
ダウンロードした動画をPCで再生すると、モデルがどこに反応しているのか確認することができます。\
緑色の線：人の操作。青色の線：AIの判断。

In [ ]:
from google.colab import files

files.download('/content/mycar/'+mp4_path)

# 学習結果の転送
学習結果は、/content/mycar/models/mypilot.tfliteに保存されます。  下記セルを実行し、tfliteファイルをダウンロードします。

RaspberryPiの~/mycar/models/mypilot.tfliteに学習結果をコピーすることで、DonkeyCarの自動走行ができるようになります。

In [ ]:
from google.colab import files

files.download('/content/mycar/models/mypilot.tflite')

# その他：ブラウザ上で動画を再生する
OpenCVで出力したmp4ファイルはここでは再生できないため、webmに変換してから再生します。  \
このffmpegを使った変換はかなり時間がかかります。


In [ ]:
webm_path = 'tub_movie.webm'
!ffmpeg -i $mp4_path -vcodec vp9 $webm_path -y -loglevel quiet

In [ ]:
import io
import base64
from IPython.display import HTML

def play_movie(webm_path):
  video = io.open(webm_path, 'r+b').read()
  encoded = base64.b64encode(video)
  return HTML(data='''<video alt="test" controls>
                      <source src="data:video/webm;base64,{0}" type="video/webm" />
                      </video>'''.format(encoded.decode('ascii')))

In [ ]:
play_movie(webm_path)